In [2]:
import pandas as pd
import os

neues_verzeichnis = "C:\\Users\\Lucian\\Desktop\\WebMining_final"
os.chdir(neues_verzeichnis)

# Datei einlesen
file_path = "steam_game_genres.csv"  # Falls nötig, Dateipfad anpassen
df_genres = pd.read_csv(file_path)
# Genres in einzelne Einträge aufsplitten und Bereinigung durchführen
df_genres['genres'] = df_genres['genres'].astype(str).fillna('')  # Sicherstellen, dass alle Werte Strings sind
all_genres = df_genres['genres'].str.split(',').explode().str.strip()  # Splitten und bereinigen

# Alle einzigartigen Genres & Varianten extrahieren
unique_genres = all_genres.unique()

# Ergebnisse anzeigen
print(f"Anzahl einzigartiger Genres: {len(unique_genres)}")
print("Einzigartige Genre-Bezeichnungen:")
for genre in sorted(unique_genres):
    print(genre)


Anzahl einzigartiger Genres: 88
Einzigartige Genre-Bezeichnungen:
Abenteuer
Acción
Accounting
Acesso Antecipado
Action
Adventure
Akció
Akcja
Aksiyon
Animation & Modeling
Audio Production
Aventura
Ação
Bağımsız Yapımcı
Casual
Design & Illustration
Early Access
Education
Estratégia
Free To Play
Game Development
Gore
Indie
Ingyenesen játszható
Korai hozzáférés
Kostenlos spielbar
MMO
Macera
Massively Multiplayer
Movie
Multijogador Massivo
Nenáročné
Nezávislé
Niezależne
Nudity
Ocazionale
Photo Editing
RPG
Racing
Rekreacyjne
Rennspiele
Rollenspiel
Sexual Content
Simulation
Simulationen
Simulatoare
Software Training
Sport
Sports
Strategy
Symulacje
Tasarım ve İllüstrasyon
Utilities
Video Production
Violent
Wczesny dostęp
Web Publishing
Wyścigowe
Yardımcı Uygulamalar
nan
Інді
Анімація та моделювання
Бесплатные
Бойовики
Вільний доступ
Дизайн та ілюстрація
Инди
Казуальные игры
Казуальні ігри
Многопользовательские игры
Пригоди
Приключенческие игры
Ранний доступ
Ролевые игры
Рольові ігри
Симуляторы

In [10]:
# Normalisierungstabelle für Genres (Übersetzungen & Synonyme)
genre_mapping = {
    "Abenteuer": "Adventure", "Aventura": "Adventure", "Macera": "Adventure", "Пригоди": "Adventure", "Приключенческие игры": "Adventure", "アドベンチャー": "Adventure",
    "Action": "Action", "Acción": "Action", "Akció": "Action", "Akcja": "Action", "Aksiyon": "Action", "Экшены": "Action", "Бойовики": "Action", "アクション": "Action", "Ação": "Action",
    "Casual": "Casual", "Kazualni": "Casual", "Casualni": "Casual", "Казуальні ігри": "Casual", "Казуальные игры": "Casual", "Nenáročné": "Casual", "Ocazionale": "Casual", "カジュアル": "Casual",
    "Early Access": "Early Access", "Acesso Antecipado": "Early Access", "Wczesny доступ": "Early Access", "Korai hozzáférés": "Early Access", "Ранний доступ": "Early Access", "Wczesny dostęp": "Early Access",
    "Free To Play": "Free-to-Play", "Kostenlos spielbar": "Free-to-Play", "Бесплатные": "Free-to-Play", "Вільний доступ": "Free-to-Play", "Ingyenesen játszható": "Free-to-Play", "無料プレイ": "Free-to-Play",
    "Indie": "Indie", "Bağımsız Yapımcı": "Indie", "Niezależне": "Indie", "Nezávislé": "Indie", "Инди": "Indie", "Інді": "Indie", "インディー": "Indie", "Niezależne": "Indie",
    "MMO": "MMO", "Massively Multiplayer": "MMO", "Multijogador Massivo": "MMO", "Многопользовательские игры": "MMO",
    "RPG": "RPG", "Rollenspiel": "RPG", "Ролевые игры": "RPG", "Рольові ігри": "RPG",
    "Simulation": "Simulation", "Simulationen": "Simulation", "Symulacje": "Simulation", "Simulatoare": "Simulation", "Симуляторы": "Simulation", "模拟": "Simulation", "シミュレーション": "Simulation",
    "Sports": "Sports", "Sport": "Sports", "Rekreacyjne": "Sports",
    "Strategy": "Strategy", "Estratégia": "Strategy", "Стратегії": "Strategy",
    "Racing": "Racing", "Rennspiele": "Racing", "Wyścigowe": "Racing",
    "Utilities": "Software & Tools", "Yardımcı Uygulamalar": "Software & Tools", "Службові програми": "Software & Tools",
    "Violent": "Violent", "Gore": "Violent",
    "Sexual Content": "Adult Content", "Nudity": "Adult Content",
    "Animation & Modeling": "Software & Tools", "Tasarım ve İllüstrasyon": "Software & Tools", "Дизайн та ілюстрація": "Software & Tools", "アニメーションとモデリング": "Software & Tools", "Анімація та моделювання": "Software & Tools",
    "Video Production": "Software & Tools", "Створення відео": "Software & Tools",
    "Audio Production": "Software & Tools", "Створення звуку": "Software & Tools",
    "Web Publishing": "Software & Tools", "Software Training": "Software & Tools", "Game Development": "Software & Tools",
    "Accounting": "Software & Tools", "Education": "Software & Tools", "Software & Tools": "Software & Tools",
    "Movie": "Movie",
    "Photo Editing": "Photo Editing",
    "Design & Illustration":"Software & Tools", "Software & Tools":"Software & Tools", "Photo Editing": "Software & Tools", "Software & Tools": "Software & Tools"
}


normalized_genres = df_genres.copy()

# Genres splitten, normalisieren und wieder zusammenfügen
normalized_genres['genres'] = normalized_genres['genres'].str.split(',')
normalized_genres = normalized_genres.explode('genres')
normalized_genres['genres'] = normalized_genres['genres'].str.strip()  
normalized_genres['genres'] = normalized_genres['genres'].fillna('Unknown')

# Normalisierung durchführen
normalized_genres['genres'] = normalized_genres['genres'].replace(genre_mapping)

# Nach Normalisierung zusammenfügen, um Duplikate zu vermeiden
normalized_genres["genres"] = normalized_genres.groupby('appid')['genres'].transform(lambda x: ', '.join(sorted(set(x))))

normalized_genres = normalized_genres.drop_duplicates(subset=['appid', 'genres'])

# Prüfen, ob alle Genres ersetzt wurden
clean_genres = normalized_genres['genres'].str.split(',').explode().str.strip().unique()

print(f"Anzahl einzigartiger Genres nach Normalisierung: {len(clean_genres)}")
print("Normalisierte Genre-Bezeichnungen:")
for genre in sorted(clean_genres):
    print(genre)

Anzahl einzigartiger Genres nach Normalisierung: 17
Normalisierte Genre-Bezeichnungen:
Action
Adult Content
Adventure
Casual
Early Access
Free-to-Play
Indie
MMO
Movie
RPG
Racing
Simulation
Software & Tools
Sports
Strategy
Violent
nan


In [4]:
normalized_genres

,appid,genres
0,570,"Action, Free-to-Play, Strategy"
1,730,"Action, Free-to-Play"
2,786520,"Early Access, Software & Tools"
3,1056640,"Action, Free-to-Play, MMO, RPG"
4,8500,"Action, Free-to-Play, MMO, RPG, Strategy"
...,...,...
19523,361160,nan
19524,1151130,"Indie, Strategy"
19525,1833490,"Adventure, Indie"
19526,1073900,Indie


In [11]:
normalized_genres.to_csv("steam_game_genres_normalized.csv", index=False)